In [1]:
from utils import *
import sys
import copy
import random
import time
from pathlib import Path
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback,CheckpointCallback,CallbackList
import torch as th
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor


In [2]:
with open('dataframes.pickle', 'rb') as handle:
    loaded_dfs = pickle.load(handle)
with open('initial_dataframes.pickle', 'rb') as handle:
    loaded_initial_dfs = pickle.load(handle)

In [ ]:
#loaded_dfs is the the dfs that has (previous_action,action) single action based logs 
print_empty_dataframes(loaded_dfs) 

print('\n\n\n')
#loaded_dfs is the the dfs that has only action one single action based logs 
print_empty_dataframes(loaded_initial_dfs) 


In [ ]:
env=transferClass(loaded_dfs,loaded_initial_dfs,'random')
evaluation_env=transferClass(loaded_dfs,loaded_initial_dfs,'random')
total_scores=0
total_scores_round=0
s = env.reset()
action_list=[]
reward_list=[]
reward_list_round=[]
done = False
while not done:
    a=env.action_space.sample()
    if a==0:
        a=1
    s_next, r, done, info = env.step(a)
    action_list.append(a)
    reward_list.append(r)
    total_scores += r
    total_scores_round += round(r,3)
    reward_list_round.append(round(r,3))
    s = s_next    
env.close()
print(f"Total Reward: {total_scores}")
print(f"Total Reward round: {total_scores_round}")
print(f"actions {action_list},   {len(action_list)}")
print(f"rewards {reward_list},  {len(reward_list)}")
print(f"rewards round {reward_list_round},  {len(reward_list_round)}")


In [ ]:
env=transferClass(loaded_dfs,loaded_initial_dfs,'random')
evaluation_env=transferClass(loaded_dfs,loaded_initial_dfs,'random')

policy_kwargs = dict(activation_fn=th.nn.ReLU,net_arch=[{'pi': [128, 128], 'vf': [128, 128]}])
model = PPO("MlpPolicy", env=env, policy_kwargs=policy_kwargs, verbose=1,tensorboard_log="./ppo_tensorboard/",ent_coef=0.01)
eval_callback = EvalCallback(evaluation_env, best_model_save_path='./ppo/ppo_best_model/',
                               log_path='./ppo/ppo_logs/', eval_freq=1000,
                               deterministic=True, render=False)
# Callback for saving checkpoints every 1000 timesteps
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./ppo/ppo_checkpoints/',
                                           name_prefix='ppo_model')

# Combine both callbacks
callback = CallbackList([checkpoint_callback, eval_callback])
model.learn(total_timesteps=1000000, callback=callback)

In [5]:
model = PPO.load("./ppo/ppo_best_model/best_model.zip")
done = False
episode_reward = 0
env=transferClass(loaded_dfs,loaded_initial_dfs,'random')
action_list=[]
reward_list=[]
obs = env.reset()
while not done:
    action, _ = model.predict(obs, deterministic=True)
    action_list.append(int(action))
#     print("action: ",action)
    obs, reward, done, info = env.step(action)
#     obs = obs.astype(np.float32)
#     print("obs: ", obs,".... reward: ",reward)
    reward_list.append(reward)
    episode_reward += reward

accumulator_df = pd.concat(env.obs_df)  # Add more DataFrames in the list if needed
env.close()
print(f"Episode reward: {episode_reward}")
print(f"actions {action_list},   {len(action_list)}")
print(f"rewards {reward_list},  {len(reward_list)}")
print(f"Average Throughput {accumulator_df['Throughput'].mean()}")
print(f"Total Energy {accumulator_df['Energy'].sum()}")
print(f"Total packet loss {accumulator_df['sender_lr'].sum()}")
print(accumulator_df.describe())

Episode reward: 1.4000000000000004
actions [7, 5, 6, 7, 5, 6, 7, 5, 4, 6, 7, 5, 6, 7, 5, 6, 7, 7, 5, 6],   20
rewards [1.0, 0.2, 4.2, -2.0, -3.4, 3.2, 1.2, -1.0, 1.4, -3.6, 1.0, -2.0, 2.0, 3.2, 0.2, -4.4, -0.2, 0.0, 5.0, -4.6],  20
Average Throughput 3.1104000000000003
Total Energy 6418.0
Total packet loss 0.24979500000000002
       Throughput  receiver_lr      Score         RTT     Energy   sender_lr
count  100.000000        100.0  100.00000  100.000000  100.00000  100.000000
mean     3.110400          0.0    2.71000   34.321000   64.18000    0.002498
std      2.854597          0.0    2.51177    7.042028   31.17477    0.012905
min      0.000000          0.0    0.00000    0.000000    0.00000    0.000000
25%      0.640000          0.0    1.00000   32.700000   47.00000    0.000000
50%      2.240000          0.0    2.00000   33.200000   73.00000    0.000000
75%      5.120000          0.0    4.00000   37.375000   90.00000    0.000000
max     11.200000          0.0   10.00000   48.500000  1

In [ ]:
from transferService import transferService
from optimizer_gd import *
from transferClass import *
import logging as log
def get_env(env_string='transferService', optimizer='ppo_simulator_trained', REMOTE_IP = "129.114.109.104", REMOTE_PORT = "80", INTERVAL = 1,INTERFACE = "eno1",SERVER_IP = '127.0.0.1',SERVER_PORT = 8080):
    for handler in log.root.handlers[:]:
        log.root.removeHandler(handler)
    log_FORMAT = '%(created)f -- %(levelname)s: %(message)s'
    extraString="logFile"
    # Create the directory if it doesn't exist
    directory = f"./logFileDir/{optimizer}/"
    os.makedirs(directory, exist_ok=True)
    log_file = f"logFileDir/{optimizer}/{optimizer}_{extraString}_{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"
    log.basicConfig(
        format=log_FORMAT,
        datefmt='%m/%d/%Y %I:%M:%S %p',
        level=log.INFO,
        handlers=[
            log.FileHandler(log_file),
            # log.StreamHandler()
        ]
    )
    transfer_service = transferService(REMOTE_IP, REMOTE_PORT, INTERVAL, INTERFACE, SERVER_IP, SERVER_PORT, optimizer, log)
    env = transferClass(transfer_service,optimizer)
    return env

class observationTranslator(gym.Wrapper):
    def __init__(self, env, obs_min, obs_max, reward_scale=1.0):
        super().__init__(env)
        self.obs_min = obs_min
        self.obs_max = obs_max
        self.reward_scale = reward_scale
        self.old_score = 0
        self.score_difference_positive_threshold = 2
        self.score_difference_negative_threshold = -2

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        return self.normalize_observation(observation)

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        normalized_obs = self.normalize_observation(observation)

        if reward != 1000000:
            score_difference = reward - self.old_score
            self.old_score = reward
            if score_difference > self.score_difference_positive_threshold:
                # normalized_reward = 1
                normalized_reward = max(-10, min(10, score_difference))
            elif score_difference < self.score_difference_negative_threshold:
                # normalized_reward = -1
                normalized_reward = max(-10, min(10, score_difference))
            else:
                normalized_reward = 0
        else:
            normalized_reward = 20

        return normalized_obs, normalized_reward, done, info



In [ ]:
env_string='transferService'
env= get_env(env_string='transferService')

# # Load from disk
data = np.load('obs_stats.npz')
obs_min = data['min']
obs_max = data['max']

print("obs_min after loading", obs_min)
print("obs_max after loading", obs_max)

data = np.load('reward_stats.npz')
reward_mean = data['mean']
reward_std = data['std']

print("reward_mean after loading", reward_mean)
print("reward_std after loading", reward_std)

# env = NormalizeObservationAndRewardWrapper(env, obs_mean, obs_std, reward_scale=1.0)
env = NormalizeObservationAndRewardWrapper(env, obs_min, obs_max, reward_scale=1.0)
